# Оцениваем регрессии на данных HS0 в спецификациях:
- Сырые данные ~ ковид и ограничения + контроль на дамми
- Сырые данные ~ ковид и ограничения + контроль на дамми | инструментальные переменные

Для второй спецификации считаем бутстраповские доверительные интервалы

In [1]:
library(arrow)
library(stargazer)
library(tidyverse)
library(plotly)
library(fixest)
signif.code = signif.code = c("***" = 0.001, "**" = 0.01, "*" = 0.05, "." = 0.10)


Присоединяю пакет: 'arrow'


Следующий объект скрыт от 'package:utils':

    timestamp



Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 


Warning message:
"пакет 'tidyverse' был собран под R версии 4.4.2"
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ lubridate::duration() masks arrow::duration()
✖ dplyr::filter()       masks stats::filter()
✖ dplyr::lag()          masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message:
"пакет 'plotly' был собран под R версии 4.4.2"

Присоеди

## Сырые данные

Под каждой регрессией вывожу summary с кластеризацией ошибок. В конце блока сравниваю робастность результата по используемым ошибкам.

In [2]:
# Скачиваю данные и переназываю переменные
df = read_parquet('../data/processed_data/trade_hs0_x_covid.parquet.gzip')
# Переменная pair нужна, чтобы считать НАС стандартные ошибки
df = df %>% mutate(pair = paste(i, j))

df$deaths_i = df$deaths
df$deaths_j = df$`deaths:r_j`

df$median_age_i = df$median_age
df$median_age_j = df$`median_age:r_j`

df$average_surface_temperature_i = df$average_surface_temperature
df$average_surface_temperature_j = df$`average_surface_temperature:r_j`

df$C1M_School_closing_i = df$C1M_School_closing
df$C1M_School_closing_j = df$`C1M_School_closing:r_j`

df$population_density_i = df$population_density
df$population_density_j = df$`population_density:r_j`

df$share_1_members_i = df$share_1_members
df$share_1_members_j = df$`share_1_members:r_j`

df$H6M_Facial_Coverings_i = df$H6M_Facial_Coverings
df$H6M_Facial_Coverings_j = df$`H6M_Facial_Coverings:r_j`

df$C8EV_International_travel_controls_i = df$C8EV_International_travel_controls
df$C8EV_International_travel_controls_j = df$`C8EV_International_travel_controls:r_j`
df %>% head()

t,i,j,v,q,v19,q19,country,t:r_i,cases,⋯,C1M_School_closing_i,C1M_School_closing_j,population_density_i,population_density_j,share_1_members_i,share_1_members_j,H6M_Facial_Coverings_i,H6M_Facial_Coverings_j,C8EV_International_travel_controls_i,C8EV_International_travel_controls_j
<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2021,LUX,LBY,968.018,210.173,501.858,127.012,LUX,2021,241.1512329,⋯,1.183562,1.49315071,248.620682,4.055137,33.34,NA,2.0082192,2.827397,2.534247,2.520548
2021,RUS,SEN,1307165.125,3131615.500,789625.305,2232100.632,RUS,2021,138.1677534,⋯,1.290411,0.07671233,8.838488,89.445110,25.69,6.92,2.3150685,3.402740,2.610959,2.561644
2020,SOM,KOR,3136.611,2045.876,5608.268,4002.770,SOM,2020,0.7276796,⋯,1.584700,1.98087430,26.542530,531.109006,1.53,31.75,0.7622951,1.653005,2.101093,2.415301
2020,TUR,CRI,57792.285,98850.453,60858.013,86163.873,TUR,2020,67.6935912,⋯,2.153005,2.40437150,108.343854,98.596161,6.37,12.07,2.2732241,1.218579,2.040984,2.322404
2021,KNA,BIH,0.520,0.008,0.460,0.005,KNA,2021,164.5090411,⋯,NA,1.00000000,179.857692,63.377090,NA,11.50,NA,2.476712,NA,1.000000
2022,AZE,MLT,22060.938,39007.156,12476.490,33493.400,AZE,2022,56.3560822,⋯,2.000000,0.27945206,122.707272,1660.971875,6.77,22.64,0.6575342,2.323288,3.000000,1.263014


In [3]:
# FE (т.е. оценка на панельных данных с использованием внутригрупповой дисперсии)
fe_no_iv = feols(log(q/q19) ~ log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j| i + j + year,
 data = df)
fe_no_iv %>% summary(cluster = ~ i+j)

NOTE: 13,943 observations removed because of NA and infinite values (LHS: 625, RHS: 13,568).



OLS estimation, Dep. Var.: log(q/q19)
Observations: 56,364
Fixed-effects: i: 175,  j: 175,  year: 3
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|)
log(deaths_i)                        -0.005611   0.005950 -0.943014 0.346981
log(deaths_j)                        -0.002078   0.005677 -0.366056 0.714768
C8EV_International_travel_controls_i -0.018008   0.017993 -1.000828 0.318300
C8EV_International_travel_controls_j -0.030099   0.015274 -1.970636 0.050352
                                        
log(deaths_i)                           
log(deaths_j)                           
C8EV_International_travel_controls_i    
C8EV_International_travel_controls_j .  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.14605     Adj. R2: 0.026372
                Within R2: 1.511e-4

In [4]:
# Добавляю гетерогенность эффектов во времени
heterogenous_fe = feols(log(q/q19) ~ year/log(deaths_i) + year/log(deaths_j) +
year/C8EV_International_travel_controls_i + year/C8EV_International_travel_controls_j|
i + j ,
data = df)
heterogenous_fe %>% summary(cluster = ~ i+j)

NOTE: 13,943 observations removed because of NA and infinite values (LHS: 625, RHS: 13,568).



OLS estimation, Dep. Var.: log(q/q19)
Observations: 56,364
Fixed-effects: i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                               Estimate Std. Error   t value
year2021                                      -0.077166   0.111634 -0.691247
year2022                                      -0.098782   0.129504 -0.762775
year2020:log(deaths_i)                         0.000018   0.006617  0.002784
year2021:log(deaths_i)                        -0.000286   0.007938 -0.036045
year2022:log(deaths_i)                        -0.015095   0.008131 -1.856449
year2020:log(deaths_j)                        -0.007398   0.007293 -1.014345
year2021:log(deaths_j)                         0.002447   0.008267  0.295996
year2022:log(deaths_j)                         0.000223   0.007206  0.030998
year2020:C8EV_International_travel_controls_i -0.038958   0.035296 -1.103752
year2021:C8EV_International_travel_controls_i -0.016029   0.017505 -0.915649
year2022:C8EV_International

### Инструментальные переменные на панельных данных

In [5]:
# Спецификация с фиксированными эффектами, куда я добавил инструменты, чтобы убрать потенциальную двустороннюю связь. 

fe_iv = feols(log(q/q19) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i+j|
 log(deaths_i) + log(deaths_j) ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
fe_iv %>% summary(cluster = ~ i+j)

NOTE: 14,279 observations removed because of NA and infinite values (LHS: 625, RHS: 9,802, IV: 6,067/10,661).



TSLS estimation - Dep. Var.: log(q/q19)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(q/q19)
Observations: 56,028
Fixed-effects: year: 3,  i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|)
fit_log(deaths_i)                    -0.023478   0.025714 -0.913025  0.36249
fit_log(deaths_j)                    -0.006637   0.021853 -0.303715  0.76171
C8EV_International_travel_controls_i -0.024346   0.018960 -1.284121  0.20081
C8EV_International_travel_controls_j -0.030037   0.017049 -1.761795  0.07986
                                        
fit_log(deaths_i)                       
fit_log(deaths_j)                       
C8EV_International_travel_controls_i    
C8EV_International_travel_controls_j .  
---
Signif. codes:  0 '***' 0.001 

### Экспорт результатов в латех

In [6]:
etable(fe_no_iv, heterogenous_fe, fe_iv, vcov = ~i+j, signif.code=signif.code, fitstat=~ . + sargan.p + wh.p + ivf)

,,fe_no_iv,heterogenous_fe,fe_iv
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(q/q19),log(q/q19),log(q/q19)
2,,,,
3,log(deaths_i),-0.0056 (0.0060),,-0.0235 (0.0257)
4,log(deaths_j),-0.0021 (0.0057),,-0.0066 (0.0219)
5,C8EV_International_travel_controls_i,-0.0180 (0.0180),,-0.0244 (0.0190)
6,C8EV_International_travel_controls_j,-0.0301. (0.0153),,-0.0300. (0.0170)
7,year2021,,-0.0772 (0.1116),
8,year2022,,-0.0988 (0.1295),
9,year2020 x log(deaths_i),,1.84e-5 (0.0066),


In [7]:
etable(fe_no_iv, heterogenous_fe, fe_iv, vcov = ~i+j, signif.code=signif.code, fitstat=~ . + sargan.p + wh.p + ivf, tex=TRUE)

\begingroup
\centering
\begin{tabular}{lccc}
   \tabularnewline \midrule \midrule
   Dependent Variable: & \multicolumn{3}{c}{log(q/q19)}\\
   Model:                                                          & (1)           & (2)                   & (3)\\  
   \midrule
   \emph{Variables}\\
   log(deaths\_i)                                                  & -0.0056       &                       & -0.0235\\   
                                                                   & (0.0060)      &                       & (0.0257)\\   
   log(deaths\_j)                                                  & -0.0021       &                       & -0.0066\\   
                                                                   & (0.0057)      &                       & (0.0219)\\   
   C8EV\_International\_travel\_controls\_i                        & -0.0180       &                       & -0.0244\\   
                                                                   & (0.0180)      &             